# Treinamento de uma CNN no CIFAR 10

**Nome: Eduardo Moraes Coraça**


O objetivo deste exercício é implementar e treinar uma rede convolucional que obtenha a maior acurácia no dataset de teste do CIFAR 10.

Podem definir sua própria rede ou utilizar redes pré-treinadas, como a Resnet.

## Inicializando o Neptune

In [ ]:
! pip install neptune-client==0.9.1

     |████████████████████████████████| 215kB 10.8MB/s 
     |████████████████████████████████| 829kB 21.4MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 163kB 34.4MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 133kB 36.7MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.9.1-py2.py3-none-any.whl size=371716 sha256=98c4f09d78371fcb0bdbd56b31fbd4c409d107c8ca9176d8e5f0432d1b1a7adb
  Stored in directory: /root/.cache/pip/wheels/6e/d4/60/21005f18273566da0463587bed8c69cce7f300a404edf26df8
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=9c9ccc653da9aea085d9d446a1927ea2a2761451acbcec551a798cb1312b695f
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-cp37-none-an

In [ ]:
import neptune.new as neptune

# Insira seu api_token para logar os resultados do treino na sua conta do Neptune.
# Como obter seu API token do Neptune:
# https://docs.neptune.ai/administration/security-and-privacy/how-to-find-and-set-neptune-api-token

run = neptune.init(project='eduardocoraca/aula6', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmMzZmYmY3MC01NWJjLTQ1MjItYmVjMC0zNGNjNGZiMDA4YzcifQ==')

https://app.neptune.ai/eduardocoraca/aula6/e/A6-6


## Importação das bibliotecas

In [ ]:
%matplotlib inline
import numpy as np

import torch
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import MNIST

torch.manual_seed(123)

## Download do Dataset

In [ ]:
!mkdir ./data

transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)



Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Dataset e dataloader

In [ ]:
train_size = 40000
val_size = 10000
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [ ]:
batch_size = 50

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print('Número de minibatches de trenamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))
print('Número de minibatches de teste:', len(test_dataloader))

x_train, y_train = next(iter(train_dataloader))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))

Número de minibatches de trenamento: 800
Número de minibatches de validação: 200
Número de minibatches de teste: 200

Dimensões dos dados de um minibatch: torch.Size([50, 3, 32, 32])
Valores mínimo e máximo dos pixels:  tensor(-1.) tensor(1.)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>


In [ ]:
# Use True, para usar o pytorch lightning original
pl_original = False

## Usando o Pytorch Lightining "SuperLight" (criado apenas para o curso).

Criamos um Pytorch Lightning "básico" que esperamos ser mais didático que o original pois o código é facil de entender caso ocorra algum erro.

As classes `LightningModule` e `Trainer` não precisam ser implementadas. Entretanto, para cada nova tarefa, uma classe que herda do `LightningModule` precisa ser definida e os seguintes métodos devem ser implementados:

 - \_\_init\_\_
 - forward
 - train_step
 - train_epoch_end
 - validation_step
 - validation_epoch_end
 - configure_optimizers

Os métodos `test_step` e `test_epoch_end` devem ser implementados apenas se trainer.test() for chamado.


In [ ]:
import abc


class LightningModule:
    @abc.abstractmethod
    def __init(self):
        return

    @abc.abstractmethod
    def forward(self):
        return

    @abc.abstractmethod    
    def training_step(self):
        return 

    @abc.abstractmethod    
    def training_epoch_end(self):
        return 

    @abc.abstractmethod    
    def validation_step(self):
        return 

    @abc.abstractmethod    
    def validation_epoch_end(self):
        return 

    @abc.abstractmethod    
    def test_step(self):
        return 

    @abc.abstractmethod    
    def test_epoch_end(self):
        return

    @abc.abstractmethod
    def configure_optimizers(self):
        return

In [ ]:
class Trainer():
    def __init__(self, max_epochs: int, gpus: int = 1):
        self.max_epochs = max_epochs
        dev = "cpu" 
        if gpus > 0:
            if torch.cuda.is_available(): 
                dev = "cuda:0"

        print(f'Using {dev}')
        self.device = torch.device(dev)

    def fit(self, model, train_dataloader, val_dataloader=None):
        assert isinstance(model, LightningModule)
        best_valid_loss = 10e9
        optimizers, _ = model.configure_optimizers()
        optimizer = optimizers[0]
        model.model.to(self.device)
        
        for i in range(self.max_epochs):
            outputs = []
            model.model.train()
            for batch_idx, (x_train, y_train) in enumerate(train_dataloader):
                x_train = x_train.to(self.device)
                y_train = y_train.to(self.device)
                output_dict = model.training_step((x_train, y_train), batch_idx)
                loss = output_dict['loss']
                # zero, backpropagation, ajusta parâmetros pelo gradiente descendente
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                outputs.append(output_dict)

            model.training_epoch_end(outputs=outputs)
            
            # Laço de Validação, um a cada época.
            if val_dataloader:
                output_val_end = self.validate(model, val_dataloader)
                print(f'Epoch {i} - {output_val_end["progress_bar"]}')
                # Salvando o melhor modelo de acordo com a loss de validação.
                if output_val_end['valid_loss'] < best_valid_loss:
                    torch.save(model.model.state_dict(), 'best_model.pt')
                    best_valid_loss = output_val_end['valid_loss']

    def validate(self, model, val_dataloader):
        outputs = []
        model.model.eval()
        with torch.no_grad():
            for batch_idx, (x, y) in enumerate(val_dataloader):
                x = x.to(self.device)
                y = y.to(self.device)
                output_dict = model.validation_step((x, y), batch_idx)
                outputs.append(output_dict)

        output_dict = model.validation_epoch_end(outputs=outputs)
        return output_dict

    def test(self, model, test_dataloader):
        outputs = []
        model.model.eval()
        with torch.no_grad():
            for batch_idx, (x, y) in enumerate(test_dataloader):
                x = x.to(self.device)
                y = y.to(self.device)
                output_dict = model.test_step((x, y), batch_idx)
                outputs.append(output_dict)

        output_dict = model.test_epoch_end(outputs=outputs)
        return output_dict

In [ ]:
if pl_original:
    !pip install pytorch_lightning
    from pytorch_lightning import LightningModule, Trainer

# Arquitetura própria

## Definindo o Modelo

In [ ]:
class Modelo(torch.nn.Module):
    def __init__(self):
        super(Modelo, self).__init__()

        # Defina seu modelo aqui.
        #self.dense = torch.nn.Sequential(
        #    torch.nn.Linear(32*32*3, 500),
        #    torch.nn.ReLU(),
        #    torch.nn.Linear(500, 10),
        #)
        self.cnn = torch.nn.Sequential(
            # camada 1
            torch.nn.Conv2d(3, 16, (3,3)), # dim: (16,30,30)
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2,2)), # dim: (16,15,15)
            # camada 2
            torch.nn.Conv2d(16, 64, (3,3)), # dim: (64,13,13)
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2,2)), # dim: (64,6,6)
            # camada 3
            torch.nn.Conv2d(64, 128, (3,3)), # dim: (128,4,4)
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2,2)) # dim: (128,2,2)
        )
        self.dense = torch.nn.Sequential(
            # dense 1
            torch.nn.Dropout(0.5),
            torch.nn.Linear(128*2*2,100),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            # dense 2
            torch.nn.Linear(100,10)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = x.reshape(-1,128*2*2)
        return self.dense(x)

In [ ]:
m = Modelo()
m.forward(x_train).shape

torch.Size([50, 10])

## Criação do modelo Pytorch Lightning

In [ ]:
class LightningClassifier(LightningModule):
    def __init__(self, hparams):
        super().__init__()

        self.hparams = hparams
        self.criterion = torch.nn.CrossEntropyLoss(reduction='none')

        # Note como a arquitetura esta dependente dos hiperparâmetros salvos.
        self.model = Modelo()

    def forward(self, x):
        logits = self.model(x)
        preds = logits.argmax(dim=1)
        return logits, preds

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        # x = x.reshape(-1, 32*32*3) passo não utilizado
        # predict da rede
        logits = self.model(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        loss = batch_losses.mean()
        run['train/batch_loss'].log(loss)

        # O PL sempre espera um retorno nomeado 'loss' da training_step.
        return {'loss': loss, 'batch_losses': batch_losses}

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()

        run['train/loss'].log(avg_loss)
        return
  
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        
        # Transforma a entrada para duas dimensões
        # x = x.reshape(-1, 32*32*3) passo não utilizado
        # predict da rede
        logits, preds = self.forward(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        # calcula a acurácia
        batch_accuracy = (preds == y)
        
        # Retornamos as losses do batch para podermos fazer a média no validation_epoch_end.
        return {'batch_losses': batch_losses, 'batch_accuracy': batch_accuracy}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()
        accuracy = torch.stack([output['batch_accuracy'] for output in outputs]).float().mean()

        run['valid/loss'].log(avg_loss)
        run['valid/acuracy'].log(accuracy)

        metrics = {'valid_loss': avg_loss.item(), 'accuracy': accuracy.item()}
        output =  {'progress_bar': metrics, 'valid_loss': avg_loss.item()}
        return output
  
    def test_step(self, val_batch, batch_idx):
        # A implementação deste método é opcional no Pytorch Lightning.
        x, y = val_batch
        
        # Transforma a entrada para duas dimensões
        # x = x.reshape(-1, 32*32*3) passo não utilizado
        # predict da rede
        logits, preds = self.forward(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        # calcula a acurácia
        batch_accuracy = (preds == y)
        
        # Retornamos as losses do batch para podermos fazer a média no validation_epoch_end.
        return {'batch_losses': batch_losses, 'batch_accuracy': batch_accuracy}

    def test_epoch_end(self, outputs):
        # A implementação deste método é opcional no Pytorch Lightning.
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()
        accuracy = torch.stack([output['batch_accuracy'] for output in outputs]).float().mean()

        run['valid/loss'].log(avg_loss)
        run['valid/acuracy'].log(accuracy)
        metrics = {'Test loss': avg_loss.item(), 'test accuracy': accuracy.item()}
        output =  {'progress_bar': metrics}
        return output

    def configure_optimizers(self):
        # Gradiente descendente
        optimizer = torch.optim.SGD(self.model.parameters(), lr=self.hparams['learning_rate'])
        # Aqui usamos um scheduler dummy pois o pytorch lightning original requer um.
        scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 1.0)
        return [optimizer], [scheduler]  # Forma de retorno para associar um otimizador a um scheduler.

## Inicialização dos parâmetros

In [ ]:
hparams = {
    'max_epochs': 100,
    'learning_rate': 0.1
}

## Treinamento

In [ ]:
pl_model = LightningClassifier(hparams=hparams)
trainer = Trainer(max_epochs=hparams['max_epochs'])
trainer.fit(pl_model, train_dataloader, val_dataloader)

## Teste

In [ ]:
trainer.test(pl_model, test_dataloader)

{'progress_bar': {'Test loss': 0.670904278755188,
  'test accuracy': 0.7680000066757202}}

# Modelo Pré-treinado

*   Para realizar Transfer Learning, consultei este material: https://pytorch.org/vision/stable/models.html

*   O dataset foi normalizado com 
```
torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
```
*   As imagens foram redimensionadas para 224 x 224

*   Estratégia: alterar a dimensão de saída da camada FC e realizar o fine-tuning da rede inteira









## Transformando o dataset

In [ ]:
# Criando os datasets com as transformações mencionadas

normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Resize((350, 350)),
                    normalize,
        ])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=False, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=False, transform=transform)


## Criando os dataloaders

In [ ]:
train_size = 40000
val_size = 10000
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

batch_size = 50

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print('Número de minibatches de trenamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))
print('Número de minibatches de teste:', len(test_dataloader))

x_train, y_train = next(iter(train_dataloader))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))

Número de minibatches de trenamento: 800
Número de minibatches de validação: 200
Número de minibatches de teste: 200

Dimensões dos dados de um minibatch: torch.Size([50, 3, 350, 350])
Valores mínimo e máximo dos pixels:  tensor(-2.1179) tensor(2.6400)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>


## Definindo o modelo

In [ ]:
import torchvision.models as models

In [ ]:
class ModeloResnet(torch.nn.Module):
    def __init__(self):
        super(ModeloResnet, self).__init__()

        self.resnet18 = models.resnet18(pretrained=True)

        # redefinindo a camada FC
        self.resnet18.fc = torch.nn.Linear(self.resnet18.fc.in_features, 10)

    def forward(self, x):
        return self.resnet18(x)

## Modelo PyTorch Lightning

In [ ]:
class LightningClassifier(LightningModule):
    def __init__(self, hparams):
        super().__init__()

        self.hparams = hparams
        self.criterion = torch.nn.CrossEntropyLoss(reduction='none')

        # Note como a arquitetura esta dependente dos hiperparâmetros salvos.
        self.model = ModeloResnet()

    def forward(self, x):
        logits = self.model(x)
        preds = logits.argmax(dim=1)
        return logits, preds

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        # x = x.reshape(-1, 32*32*3) passo não utilizado
        # predict da rede
        logits = self.model(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        loss = batch_losses.mean()
        run['train/batch_loss'].log(loss)

        # O PL sempre espera um retorno nomeado 'loss' da training_step.
        return {'loss': loss, 'batch_losses': batch_losses}

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()

        run['train/loss'].log(avg_loss)
        return
  
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        
        # Transforma a entrada para duas dimensões
        # x = x.reshape(-1, 32*32*3) passo não utilizado
        # predict da rede
        logits, preds = self.forward(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        # calcula a acurácia
        batch_accuracy = (preds == y)
        
        # Retornamos as losses do batch para podermos fazer a média no validation_epoch_end.
        return {'batch_losses': batch_losses, 'batch_accuracy': batch_accuracy}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()
        accuracy = torch.stack([output['batch_accuracy'] for output in outputs]).float().mean()

        run['valid/loss'].log(avg_loss)
        run['valid/acuracy'].log(accuracy)

        metrics = {'valid_loss': avg_loss.item(), 'accuracy': accuracy.item()}
        output =  {'progress_bar': metrics, 'valid_loss': avg_loss.item()}
        return output
  
    def test_step(self, val_batch, batch_idx):
        # A implementação deste método é opcional no Pytorch Lightning.
        x, y = val_batch
        
        # Transforma a entrada para duas dimensões
        # x = x.reshape(-1, 32*32*3) passo não utilizado
        # predict da rede
        logits, preds = self.forward(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        # calcula a acurácia
        batch_accuracy = (preds == y)
        
        # Retornamos as losses do batch para podermos fazer a média no validation_epoch_end.
        return {'batch_losses': batch_losses, 'batch_accuracy': batch_accuracy}

    def test_epoch_end(self, outputs):
        # A implementação deste método é opcional no Pytorch Lightning.
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()
        accuracy = torch.stack([output['batch_accuracy'] for output in outputs]).float().mean()

        run['valid/loss'].log(avg_loss)
        run['valid/acuracy'].log(accuracy)
        metrics = {'Test loss': avg_loss.item(), 'test accuracy': accuracy.item()}
        output =  {'progress_bar': metrics}
        return output

    def configure_optimizers(self):
        # Gradiente descendente
        optimizer = torch.optim.SGD(self.model.parameters(), lr=self.hparams['learning_rate'])
        # Aqui usamos um scheduler dummy pois o pytorch lightning original requer um.
        scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 1.0)
        return [optimizer], [scheduler]  # Forma de retorno para associar um otimizador a um scheduler.

## Inicialização dos parâmetros

In [ ]:
hparams = {
    'max_epochs': 10,
    'learning_rate': 0.1
}

## Treinamento

In [ ]:
pl_model = LightningClassifier(hparams=hparams)
trainer = Trainer(max_epochs=hparams['max_epochs'])
trainer.fit(pl_model, train_dataloader, val_dataloader)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



Using cuda:0
Epoch 0 - {'valid_loss': 0.41539716720581055, 'accuracy': 0.8570999503135681}
Epoch 1 - {'valid_loss': 0.33451607823371887, 'accuracy': 0.8854999542236328}
Epoch 2 - {'valid_loss': 0.30036741495132446, 'accuracy': 0.9034000039100647}
Epoch 3 - {'valid_loss': 0.2998550236225128, 'accuracy': 0.9077000021934509}
Epoch 4 - {'valid_loss': 0.46621376276016235, 'accuracy': 0.8734999895095825}
Epoch 5 - {'valid_loss': 0.28368237614631653, 'accuracy': 0.9200999736785889}
Epoch 6 - {'valid_loss': 0.2805667519569397, 'accuracy': 0.9246999621391296}
Epoch 7 - {'valid_loss': 0.2512226402759552, 'accuracy': 0.9317999482154846}
Epoch 8 - {'valid_loss': 0.2658153176307678, 'accuracy': 0.9314999580383301}
Epoch 9 - {'valid_loss': 0.23619306087493896, 'accuracy': 0.9388999938964844}


## Teste

In [ ]:
trainer.test(pl_model, test_dataloader)

{'progress_bar': {'Test loss': 0.26141220331192017,
  'test accuracy': 0.936199963092804}}